In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/FYP/chest_xray_for_augmentation>.zip -d /content/extracted_files


In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix


In [ ]:
# Define paths to dataset
base_dir = '/content/extracted_files/chest_xray_for_augmentation'  # Adjust the path as needed
train_dir = os.path.join(base_dir, '/content/extracted_files/chest_xray_for_augmentation/train')
test_dir = os.path.join(base_dir, '/content/extracted_files/chest_xray_for_augmentation/test')


In [ ]:
# Define data generators
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    validation_split=0.2,
    rotation_range=20,       # Randomly rotate images
    width_shift_range=0.2,   # Randomly translate images horizontally
    height_shift_range=0.2,  # Randomly translate images vertically
    shear_range=0.2,         # Apply shear transformations
    zoom_range=0.2,          # Randomly zoom images
    horizontal_flip=True     # Randomly flip images horizontally
)

In [ ]:
# Load training and validation datasets
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='training'
)

Found 4187 images belonging to 2 classes.


In [ ]:
validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

Found 1045 images belonging to 2 classes.


In [ ]:
# Define test dataset generator
test_datagen = ImageDataGenerator(rescale=1.0/255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 624 images belonging to 2 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       4,735,104 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,828,481 (18.42 MB)

 Trainable params: 4,828,481 (18.42 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=10,  # Adjust epochs based on performance
    verbose=1
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


131/131 ━━━━━━━━━━━━━━━━━━━━ 310s 2s/step - accuracy: 0.6945 - loss: 0.6247 - val_accuracy: 0.7426 - val_loss: 0.4237
Epoch 2/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 305s 2s/step - accuracy: 0.7722 - loss: 0.3838 - val_accuracy: 0.8344 - val_loss: 0.3565
Epoch 3/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 295s 2s/step - accuracy: 0.8423 - loss: 0.3484 - val_accuracy: 0.8478 - val_loss: 0.3106
Epoch 4/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 305s 2s/step - accuracy: 0.8582 - loss: 0.3117 - val_accuracy: 0.8612 - val_loss: 0.2930
Epoch 5/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 310s 2s/step - accuracy: 0.8748 - loss: 0.3014 - val_accuracy: 0.8718 - val_loss: 0.2971
Epoch 6/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.8888 - loss: 0.2724 - val_accuracy: 0.8957 - val_loss: 0.2497
Epoch 7/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 323s 2s/step - accuracy: 0.8949 - loss: 0.2579 - val_accuracy: 0.8986 - val_loss: 0.2530
Epoch 8/10
131/131 ━━━━━━━━━━━━━━━━━━━━ 321s 2s/step - accuracy: 0.9013 - loss: 0.2496 - val_accuracy: 0.893

In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy*100:.2f}%")


20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 711ms/step - accuracy: 0.5517 - loss: 0.9286
Test Accuracy: 74.36%


In [ ]:
# Generate predictions
predictions = (model.predict(test_generator) > 0.5).astype("int32")


20/20 ━━━━━━━━━━━━━━━━━━━━ 15s 713ms/step


In [ ]:
# Confusion Matrix and Classification Report
print("Confusion Matrix")
print(confusion_matrix(test_generator.classes, predictions))


Confusion Matrix
[[ 79 155]
 [  5 385]]


In [ ]:
print("Classification Report")
print(classification_report(test_generator.classes, predictions, target_names=test_generator.class_indices.keys()))


Classification Report
              precision    recall  f1-score   support

      NORMAL       0.94      0.34      0.50       234
   PNEUMONIA       0.71      0.99      0.83       390

    accuracy                           0.74       624
   macro avg       0.83      0.66      0.66       624
weighted avg       0.80      0.74      0.70       624

